In [11]:


import pandas as pd
import numpy as np
import re
import string
import nltk
import matplotlib as plty
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sklearn
import regex as re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))


%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px

import plotly.graph_objs as go
pd.set_option('display.max_colwidth', None)

from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mitra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mitra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv("data/Sheet_1.csv")
df.dropna(inplace=True, axis=1 )
df.head(4)

,response_id,class,response_text
0,response_1,not_flagged,I try and avoid this sort of conflict
1,response_2,flagged,Had a friend open up to me about his mental addiction to weed and how it was taking over his life and making him depressed
2,response_3,flagged,"I saved a girl from suicide once. She was going to swallow a bunch of pills and I talked her out of it in a very calm, loving way."
3,response_4,not_flagged,i cant think of one really...i think i may have indirectly


In [6]:
fig = px.histogram(df, x='class', title='distribution of records')
fig.show()

Clean the data


In [8]:

def remove_stopwords(list_of_words):
    filtered_list = [w for w in list_of_words if not w.lower() in stop_words]
    return filtered_list

def check_punct(list_of_words):
    """
    look at the tokenized text. if there was any punctuation, it is redundant.
    """
    filtered_list = []
    for word in list_of_words:
        if re.findall("[()!><.,`?']", word):
            pass
        else: filtered_list.append(word)
        
    return filtered_list


In [12]:
def clean_and_tokenize(text):

    word_list = word_tokenize(text)
    word_list = remove_stopwords(word_list)
    word_list = check_punct(word_list)
    
    return word_list

In [15]:
df.loc[:, 'cleaned_word_list'] = df.loc[:, 'response_text'].apply(lambda t: clean_and_tokenize(t))

Create a data frame with count and TFIDF scores for words

In [ ]:
def count_words(tokens_arrays):
    
    count_dict = {}
    for array_ in tokens_arrays:
        for word in array_:
            try: count_dict[word] +=1
            except: count_dict[word] = 1

    # sort 
    sorted_count_dict = {k:v for k,v in sorted(count_dict.items(), key=lambda item: item[1], reverse=True)}
    
    return sorted_count_dict

In [ ]:
def get_n_key_and_value(n, dict_):

    keys = [k for (k, v) in dict_.items()][:n]
    values = [v for (k, v) in dict_.items()][:n]

    return keys, values

create TFIDF scores


In [16]:
def get_tfidf_words_and_array(df):

    vectorizer = TfidfVectorizer(stop_words='english')
    transformed_data = vectorizer.fit_transform(flagged_df).toarray()
    words = vectorizer.get_feature_names_out()
    
    return transformed_data, words

In [17]:
def create_tfidf_df(transformed_data, words):
    """
    gets an array of tfidf scores and words of the document
    then places them in a dataset
    """

    df = pd.DataFrame(data=transformed_data, columns=words).sum().sort_values().reset_index()

    col_names = ['word', 'tfidf_score_sum']
    default_col_names = df.columns

    df.rename(columns={default_col_names[i]:col_names[i] for i in range(len(col_names))}, inplace=True)

    return df